In [92]:
import pandas as pd
import urllib.request
import numpy as np

## Cleaning Monthly Data

In [80]:
ene=pd.read_csv('./ecobicidata/2021-07.csv')
ene.head(1)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha Arribo,Hora_Arribo
0,F,32.0,12263.0,147.0,30/06/2021,23:50:12,181,01/07/2021,00:00:25


In [84]:
ene["full_date_retiro"] = pd.to_datetime(ene["Fecha_Retiro"] + " " + ene["Hora_Retiro"], format="%d/%m/%Y %H:%M:%S").copy()
ene["full_date_aribo"] = pd.to_datetime(ene["Fecha Arribo"] + " " + ene["Hora_Arribo"], format="%d/%m/%Y %H:%M:%S").copy()
ene["Mes"] = ene["full_date_retiro"].dt.month

In [95]:
ene["trip_time"] = ene["full_date_aribo"]  - ene["full_date_retiro"]
ene["time_delta"] = ene["trip_time"] / np.timedelta64(1,"m")

In [96]:
ene

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha Arribo,Hora_Arribo,full_date_retiro,full_date_aribo,Mes,trip_time,time_delta
0,F,32.0,12263.0,147.0,30/06/2021,23:50:12,181,01/07/2021,00:00:25,2021-06-30 23:50:12,2021-07-01 00:00:25,6.0,0 days 00:10:13,10.216667
1,M,34.0,9705.0,142.0,30/06/2021,23:28:27,427,01/07/2021,00:00:47,2021-06-30 23:28:27,2021-07-01 00:00:47,6.0,0 days 00:32:20,32.333333
2,M,19.0,10919.0,132.0,30/06/2021,23:49:52,122,01/07/2021,00:00:57,2021-06-30 23:49:52,2021-07-01 00:00:57,6.0,0 days 00:11:05,11.083333
3,M,34.0,11080.0,134.0,30/06/2021,23:53:09,68,01/07/2021,00:01:19,2021-06-30 23:53:09,2021-07-01 00:01:19,6.0,0 days 00:08:10,8.166667
4,M,35.0,10447.0,187.0,30/06/2021,23:50:14,146,01/07/2021,00:01:22,2021-06-30 23:50:14,2021-07-01 00:01:22,6.0,0 days 00:11:08,11.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351069,M,30.0,11773.0,295.0,31/07/2021,17:01:22,319,31/07/2021,17:06:05,2021-07-31 17:01:22,2021-07-31 17:06:05,7.0,0 days 00:04:43,4.716667
351070,M,68.0,12243.0,254.0,31/07/2021,16:47:10,254,31/07/2021,18:17:53,2021-07-31 16:47:10,2021-07-31 18:17:53,7.0,0 days 01:30:43,90.716667
351071,F,55.0,9883.0,42.0,31/07/2021,22:25:53,254,31/07/2021,22:40:22,2021-07-31 22:25:53,2021-07-31 22:40:22,7.0,0 days 00:14:29,14.483333
351072,M,32.0,10381.0,115.0,31/07/2021,21:32:12,262,31/07/2021,21:49:41,2021-07-31 21:32:12,2021-07-31 21:49:41,7.0,0 days 00:17:29,17.483333


## Cleaning Station Data

In [49]:
estaciones = pd.read_csv("./ecobicidata/estaciones-de-ecobici.csv")[["id","name","districtcode","districtname","location_lat","location_lon","stationtype","punto_geo"]]

estaciones["Ciclo_Estacion_Retiro"] = estaciones["id"].copy()
estaciones["Ciclo_EstacionArribo"] = estaciones["id"].copy()
estaciones_retiro = estaciones.iloc[:,[-2,1,2,3,4,5,6,7]].rename(columns={"name":"name_retiro","districtcode":"districtcode_retiro","districtname":"districtname_retiro","location_lat":"location_lat_retiro","location_lon":"location_lon_retiro","stationtype":"stationtype_retiro","punto_geo":"punto_geo_retiro"}).copy()
estaciones_arribo = estaciones.iloc[:,[-1,1,2,3,4,5,6,7]].rename(columns={"name":"name_arribo","districtcode":"districtcode_arribo","districtname":"districtname_arribo","location_lat":"location_lat_arribo","location_lon":"location_lon_arribo","stationtype":"stationtype_arribo","punto_geo":"punto_geo_arribo"}).copy()

In [64]:
first = ene.merge(estaciones_retiro.iloc[:,[0,1,4,5,7]], on="Ciclo_Estacion_Retiro", how="left").merge(estaciones_arribo.iloc[:,[0,1,4,5,7]], on="Ciclo_EstacionArribo", how="left")